## 第二節　日射に関する地域区分と日射量

### 1．適用範囲

省略

### 2. 引用規格

省略

### 3. 用語の定義

省略

### 4．年間日射量地域区分

省略

### 5．日射量等

省略

## 付録 A 傾斜面における単位面積当たりの平均日射量

### A.1 記号及び単位

省略

### A.2 傾斜面における単位面積当たりの平均日射量の計算方法

$$\large
\displaystyle
I_{s,d,t} = \left\{ \begin{array}{ll}
    I_{D,d,t} + I_{d,d,t} & (I_{D,d,t} \geq 0)\\
    I_{d,d,t} & (I_{D,d,t} < 0)\\
    \end{array}
\right.
$$

$$\large
I_{D,d,t} = I_{DN,d,t} \{sin\ h_{d,t}\ cos\ P_{\beta} + cos\ h_{d,t}\ sin\ P_{\beta}\ cos(P_{\alpha} - A_{d,t})\}
$$

$$\large
I_{d,d,t} = I_{Sky,d,t} \frac{(1+cos\ P_{\beta})}{2}
$$

$I_{s,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの平均日射量(W/m<sup>2</sup>)  
$I_{D,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの直達日射量(W/m<sup>2</sup>)  
$I_{d,d,t}$：日付$d$の時刻$t$における傾斜面の単位面積当たりの天空放射量(W/m<sup>2</sup>)  
$I_{DN,d,t}$：日付$d$の時刻$t$における法線面直達日射量(W/m<sup>2</sup>)  
$I_{Sky,d,t}$：日付$d$の時刻$t$における水平面天空日射量(W/m<sup>2</sup>)  
$h_{d,t}$：日付$d$の時刻$t$における太陽高度(°)  
$A_{d,t}$：日付$d$の時刻$t$における太陽方位角(°)  
$P_{\alpha}$：傾斜面の方位角(°)  
$P_{\beta}$：傾斜面の傾斜角(°)  

<div style="text-align: center;font-weight: bold;">表　傾斜面の方位角</div>

<table>
<thead>
<tr><th>方位角の範囲</th><th>丸めた値</th><th>方位角の範囲</th><th>丸めた値</th></tr>
</thead>
<tbody>
<tr><th>－15度（345度）から15度</th><td>0度</td><th>165度から195度</th><td>180度</td></tr>
<tr><th>15度から45度</th><td>30度</td><th>195度から225度</th><td>210度</td></tr>
<tr><th>45度から75度</th><td>60度</td><th>225度から255度</th><td>240度</td></tr>
<tr><th>75度から105度</th><td>90度</td><th>255度から285度</th><td>270度</td></tr>
<tr><th>105度から135度</th><td>120度</td><th>285度から315度</th><td>300度</td></tr>
<tr><th>135度から165度</th><td>150度</td><th>315度から345度</th><td>330度</td></tr>
</tbody>
</table>

傾斜面の傾斜角$P_{\beta}$は、水平面からの傾斜面の角度であり、傾斜面の傾斜角の一の位を四捨五入した値とし、90度を超える場合は90度とする。